# FTGP Week 1: Cryptographic primitives

This week, we will see the basic cryptographic prihitives (hash functions, digital signatures, encryption/decryption) that are needed in blockchain systehs. We will then use these next week to build our own blockchain!

These primitives are not only useful when building a blockchain, but are also extremely useful when building financial applications.

We will be using Python, but you might need to install the following packages: `hashlib, web3py, ecdsa` and `eciespy` (https://docs.python.org/3/library/hashlib.html, https://web3py.readthedocs.io/en/latest/index.html, https://github.com/tlsfuzzer/python-ecdsa, https://github.com/ecies/py).


### Question 1: Hashing

Get the hash value of `"Blockchain"` using SHA2-256, SHA2-512, SHA3-256, and SHA3-512. 

**Hint:** These hashing functions are available in the `hashlib` package.

In [1]:
import hashlib

# encode a string to bytes, or just use b"Blockchain"
s = "Blockchain"
b = str.encode(s)

h1 = hashlib.sha256(b).hexdigest()
print(h1)
h1 = hashlib.sha256(b"Blockchain").hexdigest()
print(h1)
h2 = hashlib.sha512(b"Blockchain").hexdigest()
print(h2)
h3 = hashlib.sha3_256(b"Blockchain").hexdigest()
print(h3)
h4 = hashlib.sha3_512(b"Blockchain").hexdigest()
print(h4)

625da44e4eaf58d61cf048d168aa6f5e492dea166d8bb54ec06c30de07db57e1
625da44e4eaf58d61cf048d168aa6f5e492dea166d8bb54ec06c30de07db57e1
3a45809488fe624d1f8d5c6120079fb3e04b0bb04af938c380af64128b45ab0fb28c9e280590f5aaa78c8e419dbd6de04c150dd5b7238dbff93d8e4f1f1ff4de
94074fd5892e84da500a78e4c02ff986c38815ad4063441a1caad310e89cf709
57fb5951e6be7075d3b848c38b08deb6a88ab88619a0d1805301e1d1056e68cc76b026360b8050ec59dcfe3f8932b27c1235e393cf340d0008328b224a32ccf2


Now let's do the same using Keccak-256. Keccak-256 is the primary hash function used in the Ethereum blockchain, and so will be our hash function of choice for the rest of this unit. 

**Note:** Ethereum uses a non-standard format, so the easiest way to get the same results in Python is to use the Keccak-256 function as provided by the `web3py` library.

In [2]:
from web3 import Web3

h = Web3.keccak(b"Blockchain")
print(h.hex())

0xfa8871e962875d078135f1c5b27b0f184ab6f4dff8641dd81032226ea0ae9e8c


### Question 2: Collisions

Define a hash function `H(n, msg)` that returns the first `n` bytes of the hash of the variable `msg`.

**Hint:** The method `str.encode()` might be useful.

In [3]:
def H(n, msg):
    m1 = Web3.keccak(msg.encode())
    return m1[:n]

print(H(4, "Blockchain")) # get the bytes
print(H(4, "Blockchain").hex()) # or hex value

b'\xfa\x88q\xe9'
0xfa8871e9


Find a collision of `H(1, msg)`, `H(2, msg)`, `H(3, msg)`, `H(4, msg)`, and `H(5, msg)`. Count the number of hashes you perform before finding a collision.

**Hint:** Using a `set` data structure might be a good idea.

In [4]:
def find_collisions(length):
    hash_set = set()
    number = 0
    hash = H(length, str(number))
    while (not hash in hash_set):
        hash_set.add(hash)
        number += 1
        hash = H(length, str(number))
    print("Collision!", number, hash.hex())

for i in range(1, 6):
    find_collisions(i)

Collision! 8 0xe4
Collision! 68 0xcc14
Collision! 6228 0x507072
Collision! 15121 0x25721fb3
Collision! 2084034 0xa2bd901b8c


For `H(1, msg)`, `H(2, msg)`, `H(3, msg)` find a preimage of the corresponding hashes: `b"\x00"`, `b"\x00"*2` and `b"\x00"*3`. Essentially, this is the same as finding a bitstring that when hashed produces a hash that starts with a certain number of zeros. Count the number of hashes you perform before finding each preimage.

In [5]:
def find_preimage(length, preimage):
    number = 0
    hash = H(length, str(number))
    while (not hash == preimage):
        number += 1
        hash = H(length, str(number))
    print("Found!", number, hash)

for i in range(1, 4):
    find_preimage(i, b"\x00"*i)

Found! 54 b'\x00'
Found! 114066 b'\x00\x00'
Found! 4075895 b'\x00\x00\x00'


As you can see, finding a bitstring that results in a hash with a certain property (e.g. a number of leading zeros) can be very difficult. This is the basis of the proof-of-work (PoW) algorithm that's essential for blockchains. We will explore this further next week.

### Question 3: Signatures

Generate key pairs for ECDSA and sign the string `"Blockchain"` using this signature scheme with the generated key. Then verify the obtained signature.

**Hint:** These functions are available in the `ecdsa` package.

In [6]:
from ecdsa import SigningKey

sk = SigningKey.generate()
vk = sk.verifying_key
signature = sk.sign(b"Blockchain")
print(signature.hex())

for s in [b"Blockchain", b"Bitcoin"]:
    try:
        vk.verify(signature, s)
        print(s, "is a good signature")
    except:
        print(s, "is a bad signature")

13164be37bab928a6ecd7387dd9c2c9882ebe93415117bbedf4b60d8311effc6c50551edf2f9225a8fde34f55c7e40a0
b'Blockchain' is a good signature
b'Bitcoin' is a bad signature


### Question 4: Encryption/Decryption

Generate key pairs for ECIES and encrypt the string `"Blockchain"` to obtain a ciphertext. Then decrypt the obtained ciphertext.

**Hint:** These functions are available in the `eciespy` package.

In [7]:
from ecies.utils import generate_eth_key, generate_key
from ecies import encrypt, decrypt

key = generate_eth_key()
sk = key.to_hex()
pk = key.public_key.to_hex()

en = encrypt(pk, b"Blockchain")
print(en.hex())
de = decrypt(sk, en)
print(de)

045b4b568f71d22bd819b33343fa56a21bd1a3e070a155d10909eaf22874b0582b602e3c46636ac72f0f03f736631661eb6a574d5da75dc2e931fa813a593520d55de2c23c741ed6372b763c42ce6594c671c30b1c10adaff690d2ca08fb1b97a4428c3f6afe7d9c3d7145
b'Blockchain'


### Bonus Question 5: Merkle Tree

Merkle trees are a really important data structure that is fundamental for blockchain systems. You can find more details about Merkle trees here: https://en.wikipedia.org/wiki/Merkle_tree

Implement your own Merkle tree. Make sure you distinguish leaf nodes from non-leaf node. The code below might be helpful.

**Note:** this is hard, treat is as a software challenge to improve your programming skills rather than an essential exercise you need to complete. If you are struggling with the implementation, then it might be worthwhile to study the properties and uses of Merkle trees instead. 

**Note 2:** this will be useful later on, so make sure you build it in a way that can be adapted and reused in the future.

In [8]:
import hashlib
import math

class MerkleTree():

    def __init__(self):
        self.leaves = [] # entries are assumed to be bytes
        self.hashes = []
        self.root = None

    def add(self, entry):
        # Add entries to tree

        self.leaves += [entry]
        self.hashes += [hashlib.sha256(entry).digest()]

    def build(self):
        # Build tree computing new root

        level = math.ceil(math.log(len(self.hashes), 2)) # next power of two
        if 2**level != len(self.hashes): # if not complete tree
            for i in range(2**level - len(self.hashes)): # add empty leaves so that the tree is complete
                self.add(b"")

        for i in range((2**level)-1): # calculate the rest of the nodes
            self.hashes += [hashlib.sha256(self.hashes[2*i] + self.hashes[2*i+1]).digest()]
        
        self.root = self.hashes[-1] # set root

    def get_proof(self, entry):
        # Get membership proof for a given entry

        if entry not in self.leaves:
            return None
        else:
            degree = math.log(len(self.leaves), 2)
            index = self.leaves.index(entry)
            proof = []
            while self.hashes[index] != self.root:
                if index % 2 == 0:
                    proof += [(self.hashes[index+1], 'r')] # add right sibling
                else:
                    proof += [(self.hashes[index-1], 'l')] # add left sibling
                index = int(2**degree + math.floor(index/2)) # go to parent
            return proof

    def get_root(self):
        # Return the current root
        
        if self.root == None:
            return b"0"
        else:
            return self.root

Verify that your implementation produces the correct root.

In [9]:
m = MerkleTree()
m.add(b"0")
m.add(b"1")
m.add(b"2")
m.add(b"3")
m.build()
print(m.get_root().hex())

ze = hashlib.sha256(b"0").digest()
on = hashlib.sha256(b"1").digest()
tw = hashlib.sha256(b"2").digest()
th = hashlib.sha256(b"3").digest()
zeon = hashlib.sha256(ze + on).digest()
twth = hashlib.sha256(tw + th).digest()
root = hashlib.sha256(zeon + twth).digest()
print(root.hex())

assert m.get_root().hex() == root.hex() # check that the root is correct

c478fead0c89b79540638f844c8819d9a4281763af9272c7f3968776b6052345
c478fead0c89b79540638f844c8819d9a4281763af9272c7f3968776b6052345


Write a function that verifies proofs. Verify that the proofs produced by your implementation are correct

In [10]:
def verifyProof(entry, proof, root):
    hash = hashlib.sha256(entry).digest()
    for p in proof:
        if p[1] == 'r':
            hash = hashlib.sha256(hash + p[0]).digest()
        else:
            hash = hashlib.sha256(p[0] + hash).digest()
    return (hash == root)

for d in [b"0", b"1", b"2", b"3"]:
    assert verifyProof(d, m.get_proof(d), m.get_root()) # check that the proof is correct